# Dots and Boxes

In [1]:
import pyspiel

## Set up game

In [3]:
game_string = "dots_and_boxes(num_rows=2,num_cols=2)"
game = pyspiel.load_game(game_string)

In [79]:
params = game.get_parameters()
num_rows = params['num_rows']
num_cols = params['num_cols']
num_cells = (num_rows + 1) * (num_cols + 1)
num_parts = 3   # (horizontal, vertical, cell)
num_states = 3  # (empty, player1, player2)

def part2num(part):
    p = {'h': 0, 'horizontal': 0, # Who has set the horizontal line
         'v': 1, 'vertical': 1,   # Who has set the vertical line
         'c': 2, 'cell': 2}        # Who has won the cell
    return p.get(part, part)
def state2num(state):
    s = {'e': 0, 'empty': 0,
         'p1': 1, 'player1': 1,
         'p2': 2, 'player2': 2}
    return s.get(state, state)
def num2state(state):
    s = {0: 'empty', 1: 'player1', 2: 'player2'}
    return s.get(state, state)

print(f"{num_rows=}, {num_cols=}, {num_cells=}")

num_rows=2, num_cols=2, num_cells=9


## Play game

In [41]:
state = game.new_initial_state()
state

┌ ⋯ ┬ ⋯ ┐
⋮   ⋮   ⋮
├ ⋯ ┼ ⋯ ┤
⋮   ⋮   ⋮
└ ⋯ ┴ ⋯ ┘

### Available actions in the current state

In [42]:
current_player = state.current_player()
legal_actions = state.legal_actions()
legal_actions

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [43]:
for action in legal_actions:
      print("Legal action: {} ({})".format(
          state.action_to_string(current_player, action), action))


Legal action: P1(h,0,0) (0)
Legal action: P1(h,0,1) (1)
Legal action: P1(h,1,0) (2)
Legal action: P1(h,1,1) (3)
Legal action: P1(h,2,0) (4)
Legal action: P1(h,2,1) (5)
Legal action: P1(v,0,0) (6)
Legal action: P1(v,0,1) (7)
Legal action: P1(v,0,2) (8)
Legal action: P1(v,1,0) (9)
Legal action: P1(v,1,1) (10)
Legal action: P1(v,1,2) (11)


The formula to switch between action number and action description is based on incrementally counting:
- First all horizontal lines, row by row.
- Second all vertical lines, row by row.

In [44]:
action = 10
nb_hlines = (num_rows + 1) * num_cols
if action < nb_hlines:
    row = action // num_cols
    col = action % num_cols
    print(f"{action=} : (h,{row},{col})")
else:
    action2 = action - nb_hlines
    row = action2 // (num_cols + 1)
    col = action2 % (num_cols + 1)
    print(f"{action=} : (v,{row},{col})")

action=10 : (v,1,1)


### Current observation

The observation is expressed as a tensor with dimensions:

- Axis 1: Nb of cellstates: `3` (empty, player1, player2)
- Axis 2: Nb of cells: `(num_rows + 1) * (num_cols + 1)`  
  Cells are counted row-wise (thus `cell = col + row * (num_cols + 1)`
- Axis 3: Nb of cell parts: `3` (horizontal, vertical, won by)

In [83]:
def get_observation(obs_tensor, state, row, col, part):
    state = state2num(state)
    part = part2num(part)
    idx =   part \
          + (row * (num_cols + 1) + col) * num_parts  \
          + state * (num_parts * num_cells)
    return obs_tensor[idx]
def get_observation_state(obs_tensor, row, col, part, as_str=True):
    is_state = None
    for state in range(3):
        if get_observation(obs_tensor, state, row, col, part) == 1.0:
            is_state = state
    if as_str:
        is_state = num2state(is_state)
    return is_state

In [81]:
state = game.new_initial_state()

state_str1 = str(state)
obs_tensor1 = state.observation_tensor()
state.apply_action(0)
state_str2 = str(state)
obs_tensor2 = state.observation_tensor()
state.apply_action(6)
state_str3 = str(state)
obs_tensor3 = state.observation_tensor()

print(state_str3)

┌───┬ ⋯ ┐
│   ⋮   ⋮
├ ⋯ ┼ ⋯ ┤
⋮   ⋮   ⋮
└ ⋯ ┴ ⋯ ┘



In [85]:
(get_observation_state(obs_tensor1, 0, 0, 'h'),
 get_observation_state(obs_tensor3, 0, 0, 'h'))

('empty', 'player1')

In [86]:
(get_observation_state(obs_tensor1, 0, 0, 'v'),
 get_observation_state(obs_tensor3, 0, 0, 'v'))

('empty', 'player2')